In [ ]:
from misc import load, load_class_labels
import handcrafted as hc
import bag_of_words as bow

dataset_root = "./dataset/"

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 0.5
train_info = load(dataset_root + 'train_info_dirty.csv', 1, sample_rate)
val_info = load(dataset_root + 'val_info.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values

class_labels = load_class_labels(dataset_root + 'classes.txt')

In [ ]:
handcrafted = hc.extract_features(train_dir, train_image_names, [(lambda img: hc.sift(img))])

In [ ]:
from misc import unroll_arrays
prob = 0.15
voc_size = 100

unrolled = unroll_arrays(handcrafted, 0.3)
kmeans = bow.fit(unrolled, vocabulary_size=voc_size, verbose=True, n_init=1, max_iter=200)

In [ ]:
from bag_of_words import predict
bag_of_words = predict(kmeans, handcrafted)

In [ ]:
from misc import group_stats
bow_mean, bow_cov = group_stats(bag_of_words, train_labels)

In [ ]:
from image_discard import find_outliers_per_class
names, distances = find_outliers_per_class(bow_mean, bow_cov, bag_of_words, train_labels, train_image_names)

In [ ]:
from image_discard import write_discarded_images
write_discarded_images(names, class_labels, "./discarded/", train_dir)